In [1]:
import os
import json
import torch
import numpy as np
from copy import deepcopy
from sklearn.metrics import roc_auc_score

from method.group_dro import run_group_dro
from method.group_test import run_group_test
from method.group_dro_focal import run_group_dro_focal
from models import node, tabnet, tabtrans

import numpy as np
import torch
import json
from utils.data_loader import load_data

from sklearn.metrics import (
    f1_score, recall_score, precision_score,
    roc_auc_score, brier_score_loss
)

MODEL_CLASSES = {
    'node': node.NODE,
    'tabnet': tabnet.TabNet,
    #'tabtrans' : tabtrans.TabTransformer
}

METHODS = {
    'group-dro' : run_group_dro,
    'group-test' : run_group_test,
    'group-dro-focal' : run_group_dro_focal
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def load_default_config(model_type, dataset, default_dir="experiments"):
    path = os.path.join(default_dir, f"{model_type}_{dataset}.json")
    with open(path, "r") as f:
        return json.load(f), path

def save_as_default(config, path):
    with open(path, "w") as f:
        json.dump(config, f, indent=2)

def collect_predictions(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_groups = []

    with torch.no_grad():
        for batch in dataloader:
            if model.__class__.__name__.lower() != "tabtransformer":
                x, y, g, *_ = batch
                x = x.to(device)
                outputs = model(x).squeeze()
            else:
                x_cat, x_num, y, g, *_ = batch
                x_cat = x_cat.to(device)
                x_num = x_num.to(device)
                outputs = model((x_cat, x_num)).squeeze()

            preds = torch.sigmoid(outputs).cpu().numpy()

            if preds.ndim == 0:
                all_preds.extend([preds])
            else:
                all_preds.extend(preds)

            all_labels.extend(y.numpy())
            all_groups.extend(g.numpy())

    return np.array(all_labels), np.array(all_preds), np.array(all_groups)

def generate_group_dro_optuna_runner(grid, model_type, dataset, load_data, MODEL_CLASSES, METHODS,
                                     default_dir="experiments", n_trials=30):
    import optuna

    config, default_path = load_default_config(model_type, dataset)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_class = MODEL_CLASSES[model_type]

    def to_optuna_space(grid):
        space = {}
        for k, v in grid.items():
            if isinstance(v, list):
                space[k] = {"type": "categorical", "values": v}
            elif isinstance(v, dict) and "low" in v and "high" in v:
                space[k] = v
        return space

    param_space = to_optuna_space(grid)

    def optuna_objective(trial):
        trial_config = deepcopy(config)
        trial_config["method"] = 'group-dro'

        for param_name, param_def in param_space.items():
            if param_def["type"] == "categorical":
                suggested = trial.suggest_categorical(param_name, param_def["values"])
            elif param_def["type"] == "int":
                suggested = trial.suggest_int(param_name, param_def["low"], param_def["high"])
            elif param_def["type"] == "float":
                suggested = trial.suggest_float(param_name, param_def["low"], param_def["high"],
                                                 log=param_def.get("log", False))
            else:
                raise ValueError(f"Unknown param type: {param_def['type']}")

            if param_name in trial_config["model_params"]:
                trial_config["model_params"][param_name] = suggested
            elif param_name in trial_config["train_params"]:
                trial_config["train_params"][param_name] = suggested

        if "group_dro_eta" in param_space:
            trial_config["train_params"]["group_dro_eta"] = trial.suggest_float(
                "group_dro_eta", param_space["group_dro_eta"]["low"], param_space["group_dro_eta"]["high"], log=True
            )
        if "group_dro_penalty_anneal_iters" in param_space:
            trial_config["train_params"]["group_dro_penalty_anneal_iters"] = trial.suggest_categorical(
                "group_dro_penalty_anneal_iters", param_space["group_dro_penalty_anneal_iters"]["values"]
            )

        train_loader, valid_loader, test_loader, train_df = load_data(trial_config)
        model = model_class(**trial_config["model_params"]).to(device)
        method_fn = METHODS[trial_config["method"]]
        try:
            model = method_fn(model, train_loader, valid_loader, test_loader, train_df,
                               trial_config["train_params"], device, dataset, trial_config["method"])
        except ValueError as e:
            print(f"Optuna Trial failed during method execution with ValueError: {e}")
            raise
        except TypeError as e:
            print(f"Optuna Trial failed during method execution with TypeError: {e}")
            raise

        y_true, y_prob, group_ids = collect_predictions(model, valid_loader, device)

        try:
            overall_auc = roc_auc_score(y_true, y_prob)
        except ValueError:
            overall_auc = 0.0

        group_aucs = []
        for g in np.unique(group_ids):
            mask = group_ids == g
            if np.sum(mask) > 1:
                try:
                    group_auc = roc_auc_score(y_true[mask], y_prob[mask])
                    group_aucs.append(group_auc)
                except ValueError:
                    pass
        worst_group_auc = min(group_aucs) if group_aucs else 0.0

        trial.set_user_attr("config", deepcopy(trial_config))
        trial.set_user_attr("overall_auc", overall_auc)
        trial.set_user_attr("worst_group_auc", worst_group_auc)

        return worst_group_auc

    study = optuna.create_study(direction="maximize")
    study.optimize(optuna_objective, n_trials=n_trials)

    best_trial = study.best_trial
    best_config = best_trial.user_attrs["config"]
    save_as_default(best_config, default_path.replace(".json", "_optuna_best_groupdro.json"))

    train_loader, valid_loader, test_loader, train_df = load_data(best_config)
    model_class = MODEL_CLASSES[best_config["model_type"]]
    best_model = model_class(**best_config["model_params"]).to(device)
    method_fn = METHODS[best_config["method"]]
    best_group_dro_model = method_fn(best_model, train_loader, valid_loader, test_loader, train_df,
                                      best_config["train_params"], device, dataset, best_config["method"])

    return best_group_dro_model, best_config, test_loader, valid_loader

def get_probs_and_labels_from_loader(model, loader, device):
    model.eval()
    probs, labels = [], []

    is_tabtrans = model.__class__.__name__.lower() == "tabtransformer"

    with torch.no_grad():
        for batch in loader:
            if is_tabtrans:
                x_cat, x_num, y, *_ = batch
                x_cat = x_cat.to(device)
                x_num = x_num.to(device)
                x = (x_cat, x_num)
            else:
                x, y, *_ = batch
                x = x.to(device)

            output = model(x)
            
            prob = torch.sigmoid(output).flatten().cpu().numpy()
            
            if prob.ndim == 0:
                probs.extend([prob])
            else:
                probs.extend(prob)

            if y.ndim == 0:
                labels.extend([y.numpy()])
            else:
                labels.extend(y.numpy())

    return np.array(probs), np.array(labels)

def evaluate_group_metrics(model, test_loader, device, threshold):
    model.eval()
    model.to(device)

    all_logits = []
    all_preds = []
    all_labels = []
    all_groups = []

    is_tabtrans = model.__class__.__name__.lower() == "tabtransformer"

    with torch.no_grad():
        for batch in test_loader:
            if is_tabtrans:
                x_cat, x_num, y, g, *_ = batch
                x_cat = x_cat.to(device)
                x_num = x_num.to(device)
                x = (x_cat, x_num)
            else:
                x, y, g, *_ = batch
                x = x.to(device)

            outputs = model(x)  # raw logits
            probs = torch.sigmoid(outputs).flatten().cpu().numpy()
            preds = (probs > threshold).astype(int)

            all_logits.append(probs)
            all_preds.append(preds)
            all_labels.append(y.numpy())
            all_groups.append(g.numpy())

    y_prob = np.concatenate(all_logits)
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_labels)
    group_ids = np.concatenate(all_groups)
    total = len(y_true)

    # 전체 metric
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    brier = brier_score_loss(y_true, y_prob)

    print(f"전체 AUC:     {auc:.4f}")
    print(f"전체 F1-score: {f1:.4f}")
    print(f"전체 Recall:   {recall:.4f}")
    print(f"전체 Precision: {precision:.4f}")
    print(f"전체 Brier Score: {brier:.4f}")

    # 그룹별 요약
    print("\n그룹별 성능 요약:")
    print(f"{'Group':>6} | {'Ratio (%)':>9} | {'AUC':>6} | {'F1':>6} | {'Recall':>7} | {'Precision':>9}")
    print("-" * 60)

    for g in np.unique(group_ids):
        idx = group_ids == g
        group_size = np.sum(idx)
        ratio = group_size / total * 100

        y_true_g = y_true[idx]
        y_pred_g = y_pred[idx]
        y_prob_g = y_prob[idx]

        f1_g = f1_score(y_true_g, y_pred_g, zero_division=0)
        recall_g = recall_score(y_true_g, y_pred_g, zero_division=0)
        precision_g = precision_score(y_true_g, y_pred_g, zero_division=0)

        try:
            auc_g = roc_auc_score(y_true_g, y_prob_g)
        except ValueError:
            auc_g = float('nan')

        print(f"{g:>6} | {ratio:9.2f} | {auc_g:6.4f} | {f1_g:6.4f} | {recall_g:7.4f} | {precision_g:9.4f}")

def find_best_threshold_for_f1(y_prob, y_true, num_thresholds=100):
    thresholds = np.linspace(0.0, 1.0, num_thresholds)
    best_f1 = 0.0
    best_threshold = 0.5

    for t in thresholds:
        y_pred = (y_prob > t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t

    return best_threshold, best_f1

In [3]:
grid = {
    "lr": {"type": "float", "low": 5e-5, "high": 5e-2, "log": True},
    "hidden_dim": {"type": "categorical", "values": [32, 64, 128, 256, 512]},
    "batch_size": {"type": "categorical", "values": [64, 128, 256, 512]},
    "dropout": {"type": "float", "low": 0.0, "high": 0.6},
    "num_trees": {"type": "categorical", "values": [5, 10, 20, 50]},
    "depth": {"type": "categorical", "values": [2, 3, 4, 5, 6]},
    "group_dro_eta": {"type": "float", "low": 1e-4, "high": 1.0, "log": True},
}

model_type = "node"
dataset = "compas"

best_group_dro_model, best_config_groupdro, test_loader, valid_loader = generate_group_dro_optuna_runner(
    grid, model_type, dataset, load_data, MODEL_CLASSES, METHODS, default_dir="experiments", n_trials=30
)

print("\n[Optuna Best Group-DRO 모델 설정]")
print(best_config_groupdro)

print("\n[Optuna Best Group-DRO 모델] 성능 요약")
v_prob, y_valid = get_probs_and_labels_from_loader(best_group_dro_model, valid_loader, device)
threshold, _ = find_best_threshold_for_f1(v_prob, y_valid)
print(f"threshold 균형점 : {threshold:.4f}")

print("[BEST OVERALL F1 MODEL]")
evaluate_group_metrics(best_group_dro_model, test_loader, device, threshold)

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-22 14:25:08,751] A new study created in memory with name: no-name-0cc3cce5-7064-4c5e-9b52-7135e112d521


🔥 Running GroupDRO Method on cuda...


[I 2025-04-22 14:26:54,776] Trial 0 finished with value: 0.7830621384239932 and parameters: {'lr': 0.00363048327209097, 'dropout': 0.42635204429514345, 'group_dro_eta': 0.00019032654986251679}. Best is trial 0 with value: 0.7830621384239932.


🔥 Running GroupDRO Method on cuda...


[I 2025-04-22 14:29:43,327] Trial 1 finished with value: 0.5 and parameters: {'lr': 0.0373219113847179, 'dropout': 0.021200862751463, 'group_dro_eta': 0.1149818371939144}. Best is trial 0 with value: 0.7830621384239932.


...

🔥 Running GroupDRO Method on cuda...


[I 2025-04-22 16:08:54,182] Trial 28 finished with value: 0.7142857142857143 and parameters: {'lr': 0.0017604321369968092, 'dropout': 0.48939242785119663, 'group_dro_eta': 0.00023028291491838007}. Best is trial 0 with value: 0.7830621384239932.


🔥 Running GroupDRO Method on cuda...


[I 2025-04-22 16:11:34,665] Trial 29 finished with value: 0.774679562619493 and parameters: {'lr': 0.025935565282940477, 'dropout': 0.39302880297713816, 'group_dro_eta': 0.0005763493064156883}. Best is trial 0 with value: 0.7830621384239932.


🔥 Running GroupDRO Method on cuda...

[Optuna Best Group-DRO 모델 설정]
{'dataset': 'compas', 'model_type': 'node', 'method': 'group-dro', 'model_params': {'input_dim': 34, 'hidden_dim': 64, 'num_trees': 7, 'depth': 2, 'num_classes': 1}, 'train_params': {'epochs': 30, 'lr': 0.00363048327209097, 'group_dro_eta': 0.00019032654986251679}}

[Optuna Best Group-DRO 모델] 성능 요약
threshold 균형점 : 0.3636
[BEST OVERALL F1 MODEL]
전체 AUC:     0.7875
전체 F1-score: 0.7111
전체 Recall:   0.7917
전체 Precision: 0.6455
전체 Brier Score: 0.1851

그룹별 성능 요약:
 Group | Ratio (%) |    AUC |     F1 |  Recall | Precision
------------------------------------------------------------
     0 |     53.53 | 0.7708 | 0.7429 |  0.8493 |    0.6602
     1 |      0.44 | 0.9750 | 0.7143 |  0.6250 |    0.8333
     2 |     33.23 | 0.7814 | 0.6708 |  0.7066 |    0.6385
     3 |      8.00 | 0.7828 | 0.5981 |  0.6889 |    0.5284
     4 |      0.21 | 0.8333 | 0.7143 |  0.8333 |    0.6250
     5 |      4.59 | 0.7953 | 0.6528 |  0.7000 |    0.6

In [1]:
print("\n[Optuna Best Group-DRO 모델 설정]")
print(best_config_groupdro)

print("\n[Optuna Best Group-DRO 모델] 성능 요약")
v_prob, y_valid = get_probs_and_labels_from_loader(best_group_dro_model, valid_loader, device)
threshold, _ = find_best_threshold_for_f1(v_prob, y_valid)
print(f"threshold 균형점 : {threshold:.4f}")

print("[BEST OVERALL F1 MODEL]")
evaluate_group_metrics(best_group_dro_model, test_loader, device, threshold)


[Optuna Best Group-DRO 모델 설정]


NameError: name 'best_config_groupdro' is not defined